In [ ]:
# This reloads python code imported from *.py files automatically
%reload_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import tqdm 
import tqdm.notebook
from optimization import *
import time
import multiprocessing


In [ ]:
n = 100
n_iter = 10000
distr = [G1, G2]
lambdas = [0.1, 0.6, 1, 1.7]
n_cities = 20
n_seeds = 1


def beta1(i):
    return np.log(i+1.0)


def beta2(i):
    p1 = 0.1
    p50 = 0.00001
    beta0 = -np.log(p1)
    beta1 = -np.log(p50)
    return beta0 * (beta1 / beta0) ** (i/(n-1.0))


def beta3(i):
    return np.log(i+1.0)*2


def beta4(i):
    return np.log(i+1.0)*5


mutation_strategies = [1]
init_prob = 0.0
betas = [1, 2, 5, 50, beta1, beta2]
# betas = [4, beta1, beta2, beta3, beta4]
betas = [2, 4, 5, 10, beta1, beta2, beta3]

# Evaluate over init_prob, beta

results = {}

total_runs = len(distr) * len(lambdas) * n_cities * n_seeds * len(mutation_strategies) * len(betas)

tasks = []
for d in distr:
    results[d.__name__] = {}
    ld = results[d.__name__]

    for lambd in lambdas:
        md = {}
        ld[lambd] = md
        for mutation in mutation_strategies:
            md[mutation] = {}
            
        for beta in betas:
            for mutation in mutation_strategies:
                md[mutation][beta] = []
            
            # Generate a current cities and different seeds
            for s in range(n_cities):
                np.random.seed(s)
                cities = d(n)
                for seed in range(n_seeds):
                    for mutation in mutation_strategies:
                        idx = n_seeds * s + seed
                        np.random.seed(idx)
                        args = {'cities': cities,
                                'l': lambd, 
                                'beta': beta,
                                'n_iter':n_iter,
                                'mutation_strategy': mutation, 
                                'initial_selection_probability': init_prob, 
                                'verbose': False}
                        md[mutation][beta].append(None)
                        tasks.append((md[mutation][beta], len(md[mutation][beta]) - 1, idx, args))

def task(i):
    ls, idx, seed, args = tasks[i]
    np.random.seed(seed)
    loss = optimize(**args)[-1]
    return loss
    
with multiprocessing.Pool(12) as p:
    with tqdm.notebook.tqdm(range(total_runs)) as pbar:
        losses = []
        for i, l in enumerate(p.imap_unordered(task, range(total_runs))):
            pbar.update()
            losses.append(l)
                

for i in range(len(tasks)):
    tasks[i][0][tasks[i][2]] = losses[i]

    

In [ ]:
# Plot results: Plot for each lambda the different mutation strategies
# for different values of beta
beta_str = [b.__name__ if callable(b) else b for b in betas]

fig, ax = plt.subplots( len(lambdas), len(distr), figsize=( len(distr) * 6, len(lambdas) * 4))

beta_str

for i, d in enumerate(distr):

    for j, l in enumerate(lambdas):

        for beta_idx, beta in enumerate(betas):

            means = np.zeros(len(mutation_strategies))
            std = np.zeros(len(mutation_strategies))
            for k, m in enumerate(mutation_strategies):
                means[k] = np.mean(results[d.__name__][l][m][beta])
                std[k] = np.std(results[d.__name__][l][m][beta])
        
            if len(lambdas) == 1: 
                curr_ax = ax[i]
            elif len(distr) == 1: 
                curr_ax = ax[j]
            else:
                curr_ax = ax[j, i]

            curr_ax.errorbar(np.arange(len(mutation_strategies)) + beta_idx /
                             len(betas) * 0.1, means, std[k], linestyle='None', marker='^')
#             plt.errorbar(x, y, e, linestyle='None', marker='^')
            curr_ax.set_title(f'{d.__name__}, $\lambda = {l}$')
            curr_ax.legend(beta_str, title='beta')
            curr_ax.set_xticks(np.arange(len(mutation_strategies)))
            curr_ax.set_xlabel("Mutation strategy")
            curr_ax.set_ylabel("Average loss (lower is better)")

fig.tight_layout(pad=2)
plt.savefig("parameter_evaluation3.pdf", dpi=300)